In [13]:
# imports
import sqlalchemy
from sqlalchemy import create_engine, Column, String, Integer, Numeric, MetaData, Table, type_coerce, ForeignKey, case
from sqlalchemy.orm import mapper, create_session, relationship, aliased, Session
from sqlalchemy.ext.declarative import declarative_base
import csv
from sqlalchemy import case
import numpy as np
from sqlalchemy.ext.automap import automap_base
import matplotlib.pyplot as plt
import sqlalchemy.types as types
import numbers
from sqlalchemy.sql import and_, or_, not_, func
from sqlalchemy.sql import select
import os
import glob
import re
from os.path import isfile
import gsw

import createDBfromDFO_OPDB
%matplotlib inline

In [2]:
# definitions
basepath='/ocean/eolson/MEOPAR/obs/'
basedir=basepath + 'DFOOPDB/'
dbname='DFO_OcProfDB'

In [3]:
# if PRISM.sqlite does not exist, run script to create it
if not isfile(basedir + dbname + '.sqlite'):
    import createDBfromDFO_OPDB
    createDBfromDFO_OPDB.main()
    print('done')
else:
    print('file exists')

file exists


In [4]:
Base = automap_base()
engine = create_engine('sqlite:///' + basedir + dbname + '.sqlite')
# reflect the tables in salish.sqlite:
Base.prepare(engine, reflect=True)
# mapped classes have been created
Station=Base.classes.StationTBL
Obs=Base.classes.ObsTBL
session = create_session(bind = engine, autocommit = False, autoflush = True)

In [5]:
Sal=case([(Obs.Salinity_Bottle!=None, Obs.Salinity_Bottle)], else_=
         case([(Obs.Salinity_T0_C0!=None, Obs.Salinity_T0_C0)], else_=
         case([(Obs.Salinity_T1_C1!=None, Obs.Salinity_T1_C1)], else_=
         case([(Obs.Salinity!=None, Obs.Salinity)], else_=
         case([(Obs.Salinity__Unknown!=None, Obs.Salinity__Unknown)], else_=Obs.Salinity__Pre1978)
        ))))
SalUnits=case([(Obs.Salinity_Bottle!=None, Obs.Salinity_Bottle_units)], else_=
         case([(Obs.Salinity_T0_C0!=None, Obs.Salinity_T0_C0_units)], else_=
         case([(Obs.Salinity_T1_C1!=None, Obs.Salinity_T1_C1_units)], else_=
         case([(Obs.Salinity!=None, Obs.Salinity_units)], else_=
         case([(Obs.Salinity__Unknown!=None, Obs.Salinity__Unknown_units)], else_=Obs.Salinity__Pre1978_units)
        ))))
SalFlag=case([(Obs.Salinity_Bottle!=None, Obs.Flag_Salinity_Bottle)], else_=
         case([(Obs.Salinity_T0_C0!=None, Obs.Flag_Salinity)], else_=
         case([(Obs.Salinity_T1_C1!=None, Obs.Flag_Salinity)], else_=
         case([(Obs.Salinity!=None, Obs.Flag_Salinity)], else_=
         case([(Obs.Salinity__Unknown!=None, Obs.Flag_Salinity)], else_=Obs.Quality_Flag_Sali)
        ))))

In [6]:
NO=case([(Obs.Nitrate_plus_Nitrite!=None, Obs.Nitrate_plus_Nitrite)], else_=Obs.Nitrate)
NOUnits=case([(Obs.Nitrate_plus_Nitrite!=None, Obs.Nitrate_plus_Nitrite_units)], else_=Obs.Nitrate_units)
NOFlag=case([(Obs.Nitrate_plus_Nitrite!=None, Obs.Flag_Nitrate_plus_Nitrite)], else_=Obs.Flag_Nitrate)
# Obs.Quality_Flag_Nitr does not match any nitrate obs
# ISUS not included in this NO; test separately

In [7]:
Tem=case([(Obs.Temperature!=None, Obs.Temperature)], else_=
         case([(Obs.Temperature_Primary!=None, Obs.Temperature_Primary)], else_=
         case([(Obs.Temperature_Secondary!=None, Obs.Temperature_Secondary)], else_=Obs.Temperature_Reversing)))
TemUnits=case([(Obs.Temperature!=None, Obs.Temperature_units)], else_=
         case([(Obs.Temperature_Primary!=None, Obs.Temperature_Primary_units)], else_=
         case([(Obs.Temperature_Secondary!=None, Obs.Temperature_Secondary_units)], 
              else_=Obs.Temperature_Reversing_units)))
TemFlag=Obs.Quality_Flag_Temp

In [8]:
Oxy=case([(Obs.Oxygen!=None, Obs.Oxygen)], else_=
         case([(Obs.Salinity_T0_C0!=None, Obs.Salinity_T0_C0)], else_=
         case([(Obs.Salinity_T1_C1!=None, Obs.Salinity_T1_C1)], else_=
         case([(Obs.Salinity!=None, Obs.Salinity)], else_=
         case([(Obs.Salinity__Unknown!=None, Obs.Salinity__Unknown)], else_=Obs.Salinity__Pre1978)
        ))))

In [9]:
qry=session.query(Obs.Oxygen, Obs.Oxygen_units, Obs.Oxygen_Dissolved, Obs.Oxygen_Dissolved_units, 
            Obs.Oxygen_Dissolved_SBE, Obs.Oxygen_Dissolved_SBE_units, Obs.Flag_Oxygen_Dissolved, 
            Obs.Quality_Flag_Oxyg).group_by(Obs.Oxygen_Dissolved_SBE_units).all()
         #   or_(
         #       Obs.Nitrate!=None, 
         #       Obs.Nitrate_plus_nitrite_ISUS!=None,
        # #       Obs.Nitrate_plus_Nitrite!=None)
        #).all()
for row in qry:
    print('%13s %10s %13s %10s %13s %10s %5s %5s' % row)
    # Obs.Quality_Flag_Oxyg refers to Obs.Oxygen
    # Obs.Flag_Oxygen_Dissolved refers to Obs.Oxygen_Dissolved or Obs.Oxygen_Dissolved_SBE
    # Obs.Oxygen_units:               mg/l or ml/l;      1 mg/l = 22.391 ml/31.998 = 0.700 ml/l
    # Obs.Oxygen_Dissolved_units:     mL/L or umol/kg;   1 μmol O2 = .022391 ml
    # Obs.Oxygen_Dissolved_SBE_units: mL/L or umol/kg;   1kg = approx 1 L water
    #                                                    1 ml/l = 103/22.391 = 44.661 μmol/l
    #                Molar volume at STP = 22.391 l
    #                Molar weight of oxygen = 31.998 g
    #                Atomic Mass of oxygen = 15.994 g/mol

 2.2600000000       ml/l          None       None          None       None  None    1.
         None       None  2.8580000000       mL/L  2.9000000000       mL/L     0  None
         None       None          None       None 124.5000000000    umol/kg  None  None


/home/eolson/anaconda3/envs/teos10/lib/python3.4/site-packages/sqlalchemy/sql/sqltypes.py:562: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  'storage.' % (dialect.name, dialect.driver))


In [15]:
class forceNumeric(types.TypeDecorator):

        impl = types.Numeric
        def process_bind_param(self, value, dialect):
            try:
                int(float(value))
                if int(float(value))==-99:
                    value=None
            except:
                value = None
            if (str(value).startswith('-99') or str(value).startswith('9999')):
                value = None
            return value

In [22]:
# get long and lat:
lattest=
qry=session.query(Station.LATITUDE, lattest).limit(50).all()

#latdeg=re.split('\s* \s*',Station.LATITUDE)[0]
#latmin=re.split('\s* \s*',Station.LATITUDE)[1]
#qry=session.query(Station.LATITUDE, latdeg, latmin, Station.LONGITUDE).limit(100).all()
for row in qry:
    print(row)

OperationalError: (sqlite3.OperationalError) no such function: forceNumeric [SQL: 'SELECT "StationTBL"."LATITUDE" AS "StationTBL_LATITUDE", forceNumeric("StationTBL"."LATITUDE") AS "forceNumeric_1" \nFROM "StationTBL"\n LIMIT ? OFFSET ?'] [parameters: (50, 0)]

In [22]:
frac=Obs.Oxygen_Dissolved_SBE/Obs.Oxygen_Dissolved
gsw.rho(SA,t,p)
gsw.SA_from_SP(SP,p,long,lat)
dens=gsw.rho(gsw.SA_from_SP(Sal,Obs.Pressure, ))
qry=session.query(frac, Obs.Oxygen_Dissolved_SBE, Obs.Oxygen_Dissolved, Tem, Sal, Obs.Pressure).filter(and_(
    Obs.Oxygen_Dissolved_units=='mL/L',
    Obs.Oxygen_Dissolved_SBE_units=='umol/kg',
    Obs.Oxygen_Dissolved!=None,
    Obs.Oxygen_Dissolved_SBE!=None)).all()

for row in qry:
    print('%13s %10s %13s %13s %13s %13s' % row)

43.8668490652 288.6000000000  6.5790000000  8.7917000000 32.4082000000  7.7000000000
42.6608187135 291.8000000000  6.8400000000  8.0572000000 32.2021000000  9.7000000000
43.4486838214 293.8000000000  6.7620000000  7.7215000000 32.3616000000 28.3000000000
43.0169242090 292.3000000000  6.7950000000  7.6122000000 32.4024000000 50.0000000000
43.0267062315 290.0000000000  6.7400000000  7.6356000000 32.4120000000 75.9000000000
46.1714285714 282.8000000000  6.1250000000  7.6980000000 32.4439000000 100.6000000000
44.7816222348 157.9000000000  3.5260000000  7.9360000000 33.4428000000 126.3000000000
42.9761042723 118.7000000000  2.7620000000  7.6274000000 33.7475000000 150.9000000000
44.0325497288 97.4000000000  2.2120000000  7.3245000000 33.8857000000 174.4000000000
43.4803224277 91.7000000000  2.1090000000  7.2365000000 33.9114000000 200.5000000000
42.5396825397 107.2000000000  2.5200000000  6.7984000000 33.9166000000 251.0000000000
41.9090909091 92.2000000000  2.2000000000  6.5875000000 33.94

/home/eolson/anaconda3/lib/python3.4/site-packages/sqlalchemy/sql/sqltypes.py:562: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  'storage.' % (dialect.name, dialect.driver))


In [13]:
qry=session.query(Obs.Quality_Flag_Sali, Obs.Salinity, Obs.Salinity_Bottle, Obs.Salinity_T0_C0, 
                  Obs.Salinity_T1_C1, Obs.Salinity__Pre1978, Obs.Salinity__Unknown, Obs.sourceFile
                 ).filter(and_(
            Obs.Quality_Flag_Sali!=None,
            Obs.Salinity__Pre1978==None
            )).all()
for row in qry:
    print('%5s %5s %13s %5s %13s %5s %5s %5s' % row)

In [10]:
qry=session.query(Obs.Salinity, Obs.Salinity_units, Obs.Salinity_Bottle, Obs.Salinity_Bottle_units, Obs.Salinity_T0_C0, 
                  Obs.Salinity_T0_C0_units, Obs.Salinity_T1_C1, Obs.Salinity_T1_C1_units, Obs.Salinity__Pre1978, 
                  Obs.Salinity__Pre1978_units, Obs.Salinity__Unknown, Obs.Salinity__Unknown_units,Obs.Flag_Salinity).filter(
                or_(
                    Obs.Salinity!=None,
                    Obs.Salinity_Bottle!=None,
                    Obs.Salinity_T0_C0!=None,
                    Obs.Salinity_T1_C1!=None,
                    Obs.Salinity__Pre1978!=None,
                    Obs.Salinity__Unknown!=None)).limit(50).all()
for row in qry:
    print(row)

(Decimal('25.8000000000'), 'PSS-78', None, None, None, None, None, None, None, None, None, None, None)
(Decimal('28.4100000000'), 'PSS-78', None, None, None, None, None, None, None, None, None, None, None)
(Decimal('28.8900000000'), 'PSS-78', None, None, None, None, None, None, None, None, None, None, None)
(Decimal('29.6000000000'), 'PSS-78', None, None, None, None, None, None, None, None, None, None, None)
(Decimal('29.9900000000'), 'PSS-78', None, None, None, None, None, None, None, None, None, None, None)
(Decimal('30.3500000000'), 'PSS-78', None, None, None, None, None, None, None, None, None, None, None)
(Decimal('30.6200000000'), 'PSS-78', None, None, None, None, None, None, None, None, None, None, None)
(Decimal('30.8400000000'), 'PSS-78', None, None, None, None, None, None, None, None, None, None, None)
(Decimal('31.0500000000'), 'PSS-78', None, None, None, None, None, None, None, None, None, None, None)
(None, None, Decimal('32.2963000000'), 'PSS-78', Decimal('32.2976000000')

In [ ]:
qtest=session.query(ObsTBL).filter_by(variable='water_salinity', value=0).count()
print(qtest)
qtest=session.query(ObsTBL).filter_by(variable='water_salinity', value=None).count()
print(qtest)

In [ ]:
qNLoc=session.query(ObsTBL.cast_dbid, ObsTBL.cast_datetime, ObsTBL.cast_lat, ObsTBL.cast_lon, ObsTBL.cast_datetime, ObsTBL.depth,
    ObsTBL.value).filter(
    and_(
        ObsTBL.variable == 'nitrite_concentration',
        ObsTBL.cast_lat > 48.3,
        ObsTBL.cast_lat < 48.8,
        ObsTBL.cast_lon > -125.2,
        ObsTBL.cast_lon < -124.3,
        )).subquery()

In [ ]:
test=session.query(ObsTBL).filter_by()

In [ ]:
test=session.query(qNLoc.c.value).count()
print(test)

In [ ]:
qT=session.query(ObsTBL.cast_dbid, ObsTBL.depth, ObsTBL.value).filter(ObsTBL.variable == 'water_temperature').subquery()

In [ ]:
AqNLoc=aliased(qNLoc)
AqT=aliased(qT)

In [ ]:
qNLocT=session.query(AqNLoc.c.cast_dbid, AqNLoc.c.cast_datetime, AqNLoc.c.cast_lat, AqNLoc.c.cast_lon, AqNLoc.c.depth, AqNLoc.c.value.label('N'), 
        AqT.c.value.label('T')).filter(AqNLoc.c.cast_dbid==AqT.c.cast_dbid, AqNLoc.c.depth==AqT.c.depth).subquery()

In [ ]:
qS=session.query(Salias.cast_dbid, Salias.depth, Salias.value).filter(Salias.variable == 'water_salinity').subquery()
AqS=aliased(qS)

In [ ]:
qNLocTS=session.query(qNLocT.c.cast_dbid, qNLocT.c.cast_datetime, qNLocT.c.cast_lat, qNLocT.c.cast_lon, qNLocT.c.depth, qNLocT.c.N, 
        qNLocT.c.T, AqS.c.value.label('S')).filter(qNLocT.c.cast_dbid==AqS.c.cast_dbid, qNLocT.c.depth==AqS.c.depth).subquery()

In [ ]:
for date, depth in session.query(qNLocTS.c.cast_datetime, qNLocTS.c.depth).filter(qNLocTS.c.S==0).all():
    print(date,depth)

In [ ]:
data=session.query(qNLocTS).filter(qNLocTS.c.depth>75, qNLocTS.c.S>0).all()
NO=[]
S=[]
T=[]
for dbid, lat, lon, depth, N, Tem, Sal in data:
    S.append(Sal)
    NO.append(N)
    T.append(Tem)


In [ ]:
plt.plot(S,NO,'.')
plt.xlabel('S')
plt.ylabel('N')

In [ ]:
session.close()
engine.dispose()